### 10 Academy: Artificial Intelligence Mastery
#### Week 3 Challenge
##### A/B_Hypothesis_Testing
Ethel Cherotaw

In [1]:
import sys
import numpy as np
import os
import pandas as pd
from scipy.stats import ttest_ind, chi2_contingency
from scipy import stats
from sklearn.metrics import mean_squared_error, r2_score

data_dir=data_dir = r'E:\2017.Study\Tenx\Week-3\Data\data'
src_dir = r'E:\2017.Study\Tenx\Week-3\Insurance\W3.Insurance-Planning.AIM2\src'


sys.path.append(src_dir)
sys.path.append(data_dir)
from Modeling_utils import InsuranceModeling 
from Feuture_utils  import InsuranceDataUtils

In [ ]:
csv_file_path = r'E:\2017.Study\Tenx\Week-3\Data\data\cleaned_data.csv'
df = pd.read_csv(csv_file_path, low_memory=False)
df.head()

##### Data Processing 
Even though the data is already cleaned, let's double-check for any remaining missing values.

In [ ]:
missing_values = df.isnull().sum()
# Display columns with missing values and the count
missing_values = missing_values[missing_values > 0]
print(missing_values)

#### Feature Engineering

Feature engineering: is the process of transforming raw data into relevant information for use by machine learning models. In other words, feature engineering is the process of creating predictive model features. A feature—also called a dimension—is an input variable used to generate model predictions. Because model performance largely rests on the quality of data used during training, feature engineering is a crucial preprocessing technique that requires selecting the most relevant aspects of raw training data for both the predictive task and model type under consideration.[1]

New feature 

1. Extracting information from dates, such as the age of the vehicle from RegistrationYear.
2. Binning SumInsured and cubiccapacity

Converting continuous variables into categorical bins (e.g., "Low," "Medium," "High"). This can help models to understand different risk categories more easily.

3. Extracting Information from TransactionMonth:

features such as:

Month: Helps identify seasonality.

Quarter: Groups months into quarters.

Year: Can be useful if the data spans multiple years.

4.  Log-Transforming Skewed Features:

Features like SumInsured and CalculatedPremiumPerTerm can be highly skewed. Applying a log transformation can normalize these distributions.

In [ ]:
# Initialize and preprocess
utils = InsuranceDataUtils(df, target_column='TotalPremium')
utils.add_date_features()
utils.add_vehicle_age()
utils.combine_province_zone()
utils.apply_log_transformation()

# Get the new dataset with only selected features
selected_features_df = utils.get_selected_features_df()

# Get the new dataset with only selected features
selected_features_df = utils.get_selected_features_df()
# List unique values in the 'Province' column
selected_features_df.head()


#### Preprocessing(encoding):

Selected relevant features and preprocessed the data:
1. Categorical Features: One-hot encoded.

2. Numerical Features: Standard scaled.

Created preprocessed DataFrames for analysis.

#### Data Split 
The data was split for both the TotalClaims and TotalPremium variables. First, the process began by initializing the InsuranceDataUtils class with TotalClaims as the target variable, followed by feature preprocessing. The data was then divided into training and testing sets, and the shapes of these datasets were verified. Similarly, for TotalPremium, InsuranceDataUtils was initialized with TotalPremium as the target variable. The features were preprocessed, and the data was split into training and testing sets, with the shapes of the resulting datasets also confirmed.

##### TotalPremium

In [ ]:
# Initialize with target column for TotalPremium
encode_premium = InsuranceDataUtils(selected_features_df, target_column='TotalPremium')
encode_premium.preprocess_features()
encode_premium.split_data()
preprocessed_df = encode_premium.get_preprocessed_df()

preprocessed_df.head()
# Initialize with target column for TotalClaims
X_train_premium, X_test_premium, y_train_premium, y_test_premium = encode_premium.get_train_test_data()

##### TotalClaims

In [ ]:
encode_claims = InsuranceDataUtils(selected_features_df, target_column='TotalClaims')
encode_claims.preprocess_features()
encode_claims.split_data()
preprocessed_df = encode_claims.get_preprocessed_df()

preprocessed_df.head()
# Print the shapes to verify
X_train_claims, X_test_claims, y_train_claims, y_test_claims = encode_claims.get_train_test_data()

In [ ]:
print("Data split for TotalPremium:")
print("Training set shape:", encode_premium.X_train.shape)
print("Testing set shape:", encode_premium.X_test.shape)

print("Data split for TotalClaims:")
print("Training set shape:", encode_claims.X_train.shape)
print("Testing set shape:", encode_claims.X_test.shape)


#### Modeling (Planned)

Modeling Setup:
    
Planned to use,linear regression, DecisionTreeRegressor, RandomForestRegressor, and GradientBoostingRegressor to model the data for both TotalClaims and TotalPremium.

Metrics:
    
Will evaluate models using metrics like Mean Absolute Error (MAE), Mean Squared Error (MSE), and R-squared.

In [13]:
modeling = InsuranceModeling(
    X_train_premium, X_test_premium, y_train_premium, y_test_premium,
    X_train_claims, X_test_claims, y_train_claims, y_test_claims
)


In [ ]:
modeling.train_models()
modeling.evaluate_models()



In [ ]:
modeling.feature_importance_analysis()
modeling.shap_analysis('XGBoost')  # Replace with the desired model name
modeling.lime_analysis('XGBoost')  # Replace with the desired model name


In [ ]:
# csv_file_path = r'E:\2017.Study\Tenx\Week-3\Data\data\cleaned_data.csv'
# df = pd.read_csv(csv_file_path, low_memory=False)
# utils = InsuranceDataUtils(df,'')

# # Add features
# utils.add_date_features()
# utils.add_vehicle_age()
# utils.combine_province_zone()
# utils.apply_log_transformation()

# # Get the new dataset with only selected features
# selected_features_df = utils.get_selected_features_df()
# # List unique values in the 'Province' column
# selected_features_df.head()

In [ ]:
# encode = InsuranceDataUtils(selected_features_df,target_column='TargetVariable')
# encode.preprocess_features()
# preprocessed_df = encode.get_preprocessed_df()

# preprocessed_df.head()

In [5]:
# # Example usage:
# csv_file_path = r'E:\2017.Study\Tenx\Week-3\Data\data\cleaned_data.csv'
# df = pd.read_csv(csv_file_path, low_memory=False)

# vehicle_features = InsuranceDataUtils(df)
# df_with_age = vehicle_features.add_vehicle_age()
# df_with_age_category = vehicle_features.add_vehicle_age_category()
# df_encoded = vehicle_features.encode_vehicle_age_category()

# print(df_encoded.head())
# print(df_encoded.select_dtypes(include=['object']).columns)



In [6]:
# split = InsuranceDataUtils(df_encoded)
# X_train_premium, X_test_premium, y_train_premium, y_test_premium = split.split_data('TotalPremium')

# # Example usage for TotalClaims
# X_train_claims, X_test_claims, y_train_claims, y_test_claims = split.split_data('TotalClaims')

# print("Training and Testing sets created for both TotalPremium and TotalClaims.")

In [7]:
# insurance_modeling_premium = InsuranceModeling(df_encoded, 'TotalPremium')
# insurance_modeling_premium.train_models()
# evaluation_premium = insurance_modeling_premium.evaluate_models()
# print("Evaluation results for TotalPremium:")
# for model_name, metrics in evaluation_premium.items():
#     print(f"{model_name}: {metrics}")

# # Model training and evaluation for TotalClaims
# insurance_modeling_claims = InsuranceModeling(df_encoded, 'TotalClaims')
# insurance_modeling_claims.train_models()
# evaluation_claims = insurance_modeling_claims.evaluate_models()
# print("Evaluation results for TotalClaims:")
# for model_name, metrics in evaluation_claims.items():
#     print(f"{model_name}: {metrics}")


 Series([ ], dtype: int64): indicates that there are no missing values in our dataset. The data is already cleaned on task-1 and there are no missing values to handle.
